<a href="https://colab.research.google.com/github/AatishKumar649/byte-Pair-Encoding/blob/main/Byte_Pair_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>